In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU, LSTM
from keras.optimizers import SGD
from keras.metrics import MeanSquaredError 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import Adagrad
from keras.optimizers import Adam
from keras.optimizers import Nadam

In [2]:
all_data1=pd.read_csv('모델 테스트.csv')
filtered_all_data1 = all_data1[all_data1['Datetime'].str.contains('09:00:00\+09:00')]
mean_v = np.mean(filtered_all_data1['Volume'].values)
all_data1.loc[all_data1['Volume'] == 0, 'Volume'] = mean_v
print('Number of rows and columns:', all_data1.shape)
all_data1["Datetime"] = pd.to_datetime(all_data1["Datetime"]).dt.strftime('%Y-%m-%d %H:%M:%S')
all_data1['Datetime'] = pd.to_datetime(all_data1['Datetime'])
all_data1.set_index('Datetime', inplace=True)
all_data1.drop(columns = 'Unnamed: 0', inplace = True)
all_data1.head(5) # volume = 거래량

Number of rows and columns: (2996, 8)


C:\Users\woora\AppData\Local\Temp\ipykernel_28984\57204404.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2062081.5181451612' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_data1.loc[all_data1['Volume'] == 0, 'Volume'] = mean_v


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-04-06 09:00:00,68600.0,68800.0,68600.0,68700.0,68700.0,2.062082e+06
2022-04-06 10:00:00,68600.0,68800.0,68600.0,68700.0,68700.0,1.846582e+06
2022-04-06 11:00:00,68700.0,68700.0,68600.0,68600.0,68600.0,9.013920e+05
2022-04-06 12:00:00,68700.0,68700.0,68500.0,68600.0,68600.0,2.053045e+06
2022-04-06 13:00:00,68600.0,68600.0,68500.0,68600.0,68600.0,9.246880e+05


In [3]:
all_data1['change']=0
all_data1['target']=0
for i in range(len(all_data1)-1):
    all_data1.iloc[i+1,6]=all_data1.iloc[i+1,3]-all_data1.iloc[i,3]
all_data1.loc[all_data1['change']==0, 'target']=0
all_data1.loc[all_data1['change']<0, 'target']=-1
all_data1.loc[all_data1['change']>0, 'target']=1

In [4]:
all_data1

,Open,High,Low,Close,Adj Close,Volume,change,target
Datetime,,,,,,,,
2022-04-06 09:00:00,68600.0,68800.0,68600.0,68700.0,68700.0,2.062082e+06,0,0
2022-04-06 10:00:00,68600.0,68800.0,68600.0,68700.0,68700.0,1.846582e+06,0,0
2022-04-06 11:00:00,68700.0,68700.0,68600.0,68600.0,68600.0,9.013920e+05,-100,-1
2022-04-06 12:00:00,68700.0,68700.0,68500.0,68600.0,68600.0,2.053045e+06,0,0
2022-04-06 13:00:00,68600.0,68600.0,68500.0,68600.0,68600.0,9.246880e+05,0,0
...,...,...,...,...,...,...,...,...
2024-04-16 10:00:00,80300.0,80400.0,79700.0,79900.0,79900.0,6.284883e+06,-500,-1
2024-04-16 11:00:00,79800.0,80000.0,79400.0,79900.0,79900.0,4.427816e+06,0,0
2024-04-16 12:00:00,79900.0,80000.0,79600.0,79900.0,79900.0,2.234167e+06,0,0


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

features = all_data1[['Open', 'High', 'Low', 'change', 'Volume', 'Close']].values
sc = MinMaxScaler(feature_range=(0,1)) #Normalizer() #StandardScaler() RobustScaler()
data= sc.fit_transform(features)

# 데이터셋 생성
def create_dataset(data, seq_len):
    x_data, y_data = [], []
    for i in range(len(data) - seq_len):
        x_data.append(data[i:i + seq_len])
        y_data.append(data[i + seq_len,3])
    return np.array(x_data), np.array(y_data)

# 시퀀스 길이 (7일)
SEQ_LEN = 7

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = int(len(data) - train_size - val_size)

# 시퀀스 생성을 고려한 인덱스
end_train = train_size - SEQ_LEN
end_val = train_size + val_size - SEQ_LEN

X_train, y_train = create_dataset(data[:train_size], SEQ_LEN)
X_val, y_val = create_dataset(data[train_size:train_size+val_size], SEQ_LEN)
X_test, y_test = create_dataset(data[train_size+val_size:], SEQ_LEN)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# DataLoader 설정
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=False)

val_data = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(dataset=val_data, batch_size=16, shuffle=False)

test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_data, batch_size=16, shuffle=False)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization
from torch.utils.data import DataLoader, TensorDataset

# LSTM 모델 클래스
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 최적화할 목표 함수
def train_evaluate(hidden_size, dropout_prob):
    input_size = X_train_tensor.shape[2]
    output_size = 1
    hidden_size = int(hidden_size)
    model = LSTMModel(input_size, hidden_size, output_size, dropout_prob)
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.train()
    for epoch in range(10):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # 검증 데이터로 평가
    model.eval()
    val_predictions = []
    val_targets = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_predictions.extend(outputs.cpu().numpy())
            val_targets.extend(labels.cpu().numpy())

    mse = mean_squared_error(val_targets, val_predictions)
    return -mse

# 베이지안 최적화 범위 설정
pbounds = {
    'hidden_size': (32, 512),
    'dropout_prob': (0.1, 0.5)
}

optimizer = BayesianOptimization(
    f=train_evaluate,
    pbounds=pbounds,
    random_state=42
)


optimizer.maximize(init_points=5, n_iter=25)
print("Best parameters found: ", optimizer.max)


|   iter    |  target   | dropou... | hidden... |
-------------------------------------------------


c:\Users\woora\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\woora\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


| 1         | -0.005114 | 0.2498    | 488.3     |


c:\Users\woora\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\woora\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


| 2         | -0.005878 | 0.3928    | 319.4     |


...

| 28        | -0.005145 | 0.4544    | 391.6     |


c:\Users\woora\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\woora\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


| 29        | -0.005243 | 0.1533    | 506.7     |


c:\Users\woora\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\woora\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


| 30        | -0.005727 | 0.414     | 46.73     |
Best parameters found:  {'target': -0.005107658449560404, 'params': {'dropout_prob': 0.27404013625370616, 'hidden_size': 385.9647662672542}}
